#  Demand Stock Forecasting MLOps Pipeline

**Author:** Bhupal Lambodhar  
**Email:** btiduwarlambodhar@sandiego.edu  
**Repository:** https://github.com/btlambodh/demand-stock-forecasting-mlops

---

##  Overview

This notebook provides an interactive interface to run the complete MLOps pipeline for demand stock forecasting. It replicates all the functionality from the project's Makefile while preserving SageMaker context and providing comprehensive monitoring.

###  Key Features:
- **Data Pipeline Management**: Validation, feature engineering, and infrastructure setup
- **Model Training & Registry**: Automated training with SageMaker Model Registry integration
- **Deployment Workflows**: Dev, staging, and production deployment automation
- **Monitoring & Observability**: Real-time performance and drift monitoring
- **Quality Assurance**: Testing, linting, and security checks

###  Architecture Components:
- **AWS Services**: SageMaker, S3, Athena, Feature Store
- **ML Framework**: Scikit-learn with custom forecasting models
- **Infrastructure**: Feature Store for ML, Athena for analytics
- **Monitoring**: Performance tracking and data drift detection

---

##  1. Environment Setup & Configuration

**Purpose**: Initialize the environment, load configurations, and establish AWS connections.

This cell sets up all necessary imports, loads project configuration, and establishes connections to AWS services. It also performs health checks to ensure all dependencies are available.

In [1]:
import os
import sys
import yaml
import json
import boto3
import pandas as pd
import numpy as np
import subprocess
import time
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Project configuration
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
CONFIG_PATH = PROJECT_ROOT / 'config.yaml'
SRC_PATH = PROJECT_ROOT / 'src'

# Add source path to Python path
sys.path.insert(0, str(SRC_PATH))

print(f" Project Root: {PROJECT_ROOT}")
print(f" Source Path: {SRC_PATH}")
print(f" Config Path: {CONFIG_PATH}")

# Load configuration
try:
    with open(CONFIG_PATH, 'r') as f:
        config = yaml.safe_load(f)
    print(" Configuration loaded successfully")
    
    # Display key configuration
    aws_region = config['aws']['region']
    s3_bucket = config['aws']['s3']['bucket_name']
    print(f" AWS Region: {aws_region}")
    print(f" S3 Bucket: {s3_bucket}")
    
except FileNotFoundError:
    print(" Config file not found. Please ensure config.yaml exists in the project root.")
    raise

# Initialize AWS clients
try:
    s3_client = boto3.client('s3', region_name=aws_region)
    sagemaker_client = boto3.client('sagemaker', region_name=aws_region)
    athena_client = boto3.client('athena', region_name=aws_region)
    
    # Test AWS connectivity
    s3_client.list_objects_v2(Bucket=s3_bucket, MaxKeys=1)
    sagemaker_client.list_endpoints(MaxResults=1)
    athena_client.list_work_groups()
    
    print(" AWS services connected successfully")
    print("   • S3 access confirmed")
    print("   • SageMaker access confirmed")
    print("   • Athena access confirmed")
    
except Exception as e:
    print(f" AWS connection failed: {e}")
    print("Please check your AWS credentials and permissions.")

 Project Root: /home/sagemaker-user/demand-stock-forecasting-mlops
 Source Path: /home/sagemaker-user/demand-stock-forecasting-mlops/src
 Config Path: /home/sagemaker-user/demand-stock-forecasting-mlops/config.yaml
 Configuration loaded successfully
 AWS Region: us-east-1
 S3 Bucket: sagemaker-us-east-1-346761359662
 AWS connection failed: An error occurred (AccessDeniedException) when calling the ListWorkGroups operation: User: arn:aws:sts::346761359662:assumed-role/AmazonSageMaker-ExecutionRole-20250511T063988/SageMaker is not authorized to perform: athena:ListWorkGroups because no identity-based policy allows the athena:ListWorkGroups action
Please check your AWS credentials and permissions.


##  2. Project Status & Health Check

**Purpose**: Display current project status and perform comprehensive health checks.

This cell provides an overview of the current project state, including git status, data availability, model files, and system resources. It helps identify any issues before running pipelines.

In [3]:
def run_health_check():
    """Comprehensive system health check"""
    print(" SYSTEM HEALTH CHECK")
    print("=" * 50)
    
    # Python environment
    print(f" Python Version: {sys.version.split()[0]}")
    print(f" Virtual Environment: {os.environ.get('CONDA_DEFAULT_ENV', 'Not detected')}")
    
    # Git status
    try:
        git_branch = subprocess.check_output(['git', 'branch', '--show-current'], 
                                           cwd=PROJECT_ROOT, text=True).strip()
        print(f" Git Branch: {git_branch}")
        
        git_status = subprocess.check_output(['git', 'status', '--porcelain'], 
                                           cwd=PROJECT_ROOT, text=True)
        if git_status.strip():
            print(" Git Status: Uncommitted changes detected")
        else:
            print(" Git Status: Clean working directory")
    except:
        print(" Git Status: Not a git repository or git not available")
    
    # Directory structure
    data_path = PROJECT_ROOT / 'data'
    models_path = PROJECT_ROOT / 'models'
    
    print(f"\n DIRECTORY STATUS:")
    print(f"   Data directory: {'✅' if data_path.exists() else '❌'} {data_path}")
    print(f"   Models directory: {'✅' if models_path.exists() else '❌'} {models_path}")
    
    if data_path.exists():
        data_files = list(data_path.rglob('*.csv')) + list(data_path.rglob('*.parquet'))
        print(f"   Data files found: {len(data_files)}")
    
    if models_path.exists():
        model_files = list(models_path.rglob('*.pkl')) + list(models_path.rglob('*.joblib'))
        print(f"   Model files found: {len(model_files)}")
    
    # System resources
    try:
        import psutil
        cpu_percent = psutil.cpu_percent(interval=1)
        memory_percent = psutil.virtual_memory().percent
        disk_percent = psutil.disk_usage('/').percent
        
        print(f"\n SYSTEM RESOURCES:")
        print(f"   CPU Usage: {cpu_percent:.1f}%")
        print(f"   Memory Usage: {memory_percent:.1f}%")
        print(f"   Disk Usage: {disk_percent:.1f}%")
    except ImportError:
        print("\n System resource monitoring not available (psutil not installed)")
    
    # Dependency check
    print(f"\n DEPENDENCIES:")
    dependencies = [
        ('pandas', 'import pandas'),
        ('numpy', 'import numpy'),
        ('scikit-learn', 'import sklearn'),
        ('boto3', 'import boto3'),
        ('yaml', 'import yaml')
    ]
    
    for name, import_stmt in dependencies:
        try:
            exec(import_stmt)
            print(f"   ✅ {name}")
        except ImportError:
            print(f"   ❌ {name} - Missing")
    
    print("\n" + "=" * 50)
    print(" Health check completed!")

# Run health check
run_health_check()

 SYSTEM HEALTH CHECK
 Python Version: 3.12.9
 Virtual Environment: base
 Git Branch: main
 Git Status: Uncommitted changes detected

 DIRECTORY STATUS:
   Data directory: ✅ /home/sagemaker-user/demand-stock-forecasting-mlops/data
   Models directory: ✅ /home/sagemaker-user/demand-stock-forecasting-mlops/models
   Data files found: 8
   Model files found: 6

 SYSTEM RESOURCES:
   CPU Usage: 0.1%
   Memory Usage: 1.6%
   Disk Usage: 0.0%

 DEPENDENCIES:
   ✅ pandas
   ✅ numpy
   ✅ scikit-learn
   ✅ boto3
   ✅ yaml

 Health check completed!


##  3. Pipeline Utility Functions

**Purpose**: Define utility functions for running pipeline commands and monitoring progress.

These functions provide a standardized way to execute pipeline steps, handle errors, and track progress. They maintain the same functionality as the Makefile while adding enhanced logging and error handling for notebook environments.

In [4]:
def run_pipeline_step(step_name, command, description="", check_files=None, timeout=300):
    """
    Execute a pipeline step with comprehensive logging and error handling
    
    Args:
        step_name: Name of the pipeline step
        command: Command to execute (list or string)
        description: Description of what this step does
        check_files: List of files to verify after execution
        timeout: Timeout in seconds
    """
    print(f"\n EXECUTING: {step_name}")
    print(f" Description: {description}")
    print(f" Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 60)
    
    start_time = time.time()
    
    try:
        if isinstance(command, str):
            command = command.split()
        
        # Execute command
        result = subprocess.run(
            command,
            cwd=PROJECT_ROOT,
            capture_output=True,
            text=True,
            timeout=timeout
        )
        
        execution_time = time.time() - start_time
        
        if result.returncode == 0:
            print(f" {step_name} completed successfully")
            print(f" Execution time: {execution_time:.2f} seconds")
            
            # Show stdout if available
            if result.stdout.strip():
                print(f" Output:\n{result.stdout}")
            
            # Verify output files if specified
            if check_files:
                print(f"\n Verifying output files:")
                for file_path in check_files:
                    full_path = PROJECT_ROOT / file_path
                    if full_path.exists():
                        print(f"    {file_path}")
                    else:
                        print(f"    {file_path} - Not found")
            
            return True
            
        else:
            print(f" {step_name} failed with return code {result.returncode}")
            print(f" Execution time: {execution_time:.2f} seconds")
            
            if result.stderr.strip():
                print(f" Error output:\n{result.stderr}")
            if result.stdout.strip():
                print(f" Standard output:\n{result.stdout}")
            
            return False
            
    except subprocess.TimeoutExpired:
        print(f" {step_name} timed out after {timeout} seconds")
        return False
        
    except Exception as e:
        execution_time = time.time() - start_time
        print(f" {step_name} failed with exception: {e}")
        print(f" Execution time: {execution_time:.2f} seconds")
        return False

def run_python_script(script_path, args=None, description=""):
    """
    Run a Python script with arguments
    
    Args:
        script_path: Path to Python script relative to project root
        args: List of command line arguments
        description: Description of the script
    """
    command = ['python3', str(script_path)]
    if args:
        command.extend(args)
    
    script_name = Path(script_path).name
    return run_pipeline_step(
        step_name=f"Python Script: {script_name}",
        command=command,
        description=description
    )

def create_directories():
    """Create necessary project directories"""
    directories = [
        'data/raw',
        'data/processed', 
        'data/validation',
        'data/monitoring/logs',
        'data/monitoring/reports',
        'data/monitoring/metrics',
        'models',
        'reports'
    ]
    
    print(" Creating project directories...")
    for directory in directories:
        dir_path = PROJECT_ROOT / directory
        dir_path.mkdir(parents=True, exist_ok=True)
        print(f"    {directory}")
    
    print(" All directories created successfully!")

def show_pipeline_status(pipeline_name, steps_completed, total_steps, current_step=""):
    """
    Display pipeline progress
    """
    progress = (steps_completed / total_steps) * 100
    progress_bar = "" * int(progress // 5) + "" * (20 - int(progress // 5))
    
    print(f"\n {pipeline_name} Progress: [{progress_bar}] {progress:.1f}%")
    print(f" Step {steps_completed}/{total_steps}: {current_step}")
    print("-" * 60)

print(" Pipeline utility functions loaded successfully!")

 Pipeline utility functions loaded successfully!


##  4. Data Pipeline - Complete Workflow

**Purpose**: Execute the complete data pipeline including validation, feature engineering, and infrastructure setup.

This pipeline performs:
1. **Data Validation**: Comprehensive checks for data quality, schema validation, and consistency
2. **Feature Engineering**: Transform raw data into ML-ready features with time series components
3. **Feature Store Setup**: Configure SageMaker Feature Store for ML workflows
4. **Athena Integration**: Setup analytics tables for business intelligence

**Equivalent Makefile Commands**: 
- `make pipeline-data-full`
- `make pipeline-bi`

In [6]:
def run_data_pipeline_full(include_bi=True):
    """
    Execute the complete data pipeline with all integrations
    
    Args:
        include_bi: Whether to include business intelligence setup
    """
    pipeline_name = "Complete Data Pipeline"
    total_steps = 6 if include_bi else 4
    current_step = 0
    
    print(f"\n STARTING: {pipeline_name}")
    print("=" * 70)
    
    # Create necessary directories
    create_directories()
    
    # Step 1: Data Validation
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Data Validation")
    
    success = run_python_script(
        script_path='src/data_processing/data_validation.py',
        args=[
            '--config', 'config.yaml',
            '--data-path', 'data/raw/',
            '--output-path', 'data/validation/'
        ],
        description="Validate data quality, schema, and consistency"
    )
    
    if not success:
        print(" Data pipeline failed at validation step")
        return False
    
    # Step 2: Feature Engineering
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Feature Engineering")
    
    success = run_python_script(
        script_path='src/data_processing/feature_engineering.py',
        args=[
            '--config', 'config.yaml',
            '--data-path', 'data/raw/',
            '--output-path', 'data/processed/'
        ],
        description="Generate ML-ready features with time series components"
    )
    
    if not success:
        print(" Data pipeline failed at feature engineering step")
        return False
    
    # Step 3: Feature Store & Athena Setup
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Infrastructure Setup")
    
    success = run_python_script(
        script_path='src/data_processing/feature_store_integration.py',
        args=[
            '--config', 'config.yaml',
            '--data-path', 'data/processed/',
            '--all'  # Setup both Feature Store and Athena
        ],
        description="Setup SageMaker Feature Store and Athena tables"
    )
    
    if not success:
        print(" Data pipeline failed at infrastructure setup")
        return False
    
    # Step 4: Verify Athena Setup
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Athena Verification")
    
    try:
        athena_client = boto3.client('athena', region_name=config['aws']['region'])
        
        # Test query to verify tables
        query = "SHOW TABLES IN demand_stock_forecasting_mlops_feature_store"
        response = athena_client.start_query_execution(
            QueryString=query,
            ResultConfiguration={
                'OutputLocation': config['aws']['athena']['query_results_location']
            },
            WorkGroup=config['aws']['athena'].get('workgroup', 'primary')
        )
        
        print(f" Athena verification completed")
        print(f" Query ID: {response['QueryExecutionId']}")
        
    except Exception as e:
        print(f" Athena verification failed: {e}")
    
    if include_bi:
        # Step 5: Run Sample Queries (BI)
        current_step += 1
        show_pipeline_status(pipeline_name, current_step, total_steps, "Business Intelligence Queries")
        
        success = run_python_script(
            script_path='scripts/run_sample_queries.py',
            description="Execute sample BI queries for analytics validation"
        )
        
        if not success:
            print(" BI queries failed, but data pipeline is functional")
    
    # Final status
    current_step = total_steps
    show_pipeline_status(pipeline_name, current_step, total_steps, "Completed")
    
    print(f"\n {pipeline_name} completed successfully!")
    print("\n NEXT STEPS:")
    print("   1.  Open AWS Athena Console")
    print("   2.  Select 'demand_stock_forecasting_mlops_feature_store' database")
    print("   3.  Run: SHOW TABLES")
    print("   4.  Query features: SELECT * FROM features_complete LIMIT 10")
    print("   5.  Ready for model training!")
    
    return True

# Execute the complete data pipeline
print(" Ready to run complete data pipeline...")
print(" This includes: validation → feature engineering → infrastructure setup → BI")
print("\n Click 'Run' to start the pipeline or set run_pipeline=False to skip")

# Set this to True to run the pipeline
run_pipeline = True

if run_pipeline:
    pipeline_success = run_data_pipeline_full(include_bi=True)
    if pipeline_success:
        print("\n Data pipeline execution completed successfully!")
    else:
        print("\n Data pipeline execution failed. Check logs for details.")
else:
    print(" Pipeline execution skipped. Set run_pipeline=True to execute.")

 Ready to run complete data pipeline...
 This includes: validation → feature engineering → infrastructure setup → BI

 Click 'Run' to start the pipeline or set run_pipeline=False to skip

 STARTING: Complete Data Pipeline
 Creating project directories...
    data/raw
    data/processed
    data/validation
    data/monitoring/logs
    data/monitoring/reports
    data/monitoring/metrics
    models
    reports
 All directories created successfully!

 Complete Data Pipeline Progress: [] 16.7%
 Step 1/6: Data Validation
------------------------------------------------------------

 EXECUTING: Python Script: data_validation.py
 Description: Validate data quality, schema, and consistency
 Started at: 2025-06-23 15:36:34
------------------------------------------------------------
 Python Script: data_validation.py completed successfully
 Execution time: 1.82 seconds
 Output:

DATA VALIDATION SUMMARY
Files Validated: 4
Overall Quality Score: 97.49/100
Validation Status: PASSED


 Complete Data

##  5. Model Training & Registry Pipeline

**Purpose**: Train machine learning models and register them to SageMaker Model Registry.

This pipeline performs:
1. **Model Training**: Train multiple forecasting models with cross-validation
2. **Model Evaluation**: Comprehensive performance evaluation and comparison
3. **Model Registration**: Register best models to SageMaker Model Registry
4. **Version Management**: Automatic versioning and metadata tracking

**Features**:
- Automated hyperparameter tuning
- Multiple model architectures (ARIMA, Random Forest, XGBoost)
- Performance metrics calculation
- Model artifact management

**Equivalent Makefile Commands**: 
- `make train-models`
- `make register-models`
- `make pipeline-train-full`

In [7]:
def run_training_pipeline():
    """
    Execute the complete model training and registration pipeline
    """
    pipeline_name = "Model Training & Registry Pipeline"
    total_steps = 4
    current_step = 0
    
    print(f"\n STARTING: {pipeline_name}")
    print("=" * 70)
    
    # Generate model version
    model_version = f"v{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    print(f" Model Version: {model_version}")
    
    # Step 1: Model Training
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Model Training")
    
    success = run_python_script(
        script_path='src/training/train_model.py',
        args=[
            '--config', 'config.yaml',
            '--data-path', 'data/processed/',
            '--output-path', 'models/',
            '--model-version', model_version,
            '--forecast-horizon', '1'
        ],
        description="Train forecasting models with evaluation"
    )
    
    if not success:
        print(" Training pipeline failed at model training step")
        return False
    
    # Step 2: Verify Model Artifacts
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Model Verification")
    
    models_path = PROJECT_ROOT / 'models'
    
    # Check for required model files
    required_files = [
        'best_model.pkl',
        'evaluation.json'
    ]
    
    print(" Verifying model artifacts:")
    all_files_exist = True
    
    for file_name in required_files:
        file_path = models_path / file_name
        if file_path.exists():
            file_size = file_path.stat().st_size
            print(f"    {file_name} ({file_size:,} bytes)")
        else:
            print(f"    {file_name} - Missing")
            all_files_exist = False
    
    if not all_files_exist:
        print(" Some model artifacts are missing")
        return False
    
    # Load and display evaluation metrics
    try:
        eval_path = models_path / 'evaluation.json'
        with open(eval_path, 'r') as f:
            evaluation = json.load(f)
        
        print("\n Model Performance Metrics:")
        for metric, value in evaluation.items():
            if isinstance(value, (int, float)):
                print(f"   • {metric}: {value:.4f}")
            else:
                print(f"   • {metric}: {value}")
    
    except Exception as e:
        print(f" Could not load evaluation metrics: {e}")
    
    # Step 3: Model Registration
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Model Registration")
    
    success = run_python_script(
        script_path='src/deployment/model_registry.py',
        args=[
            '--config', 'config.yaml',
            '--action', 'register',
            '--models-dir', 'models',
            '--evaluation-file', 'models/evaluation.json'
        ],
        description="Register trained models to SageMaker Model Registry"
    )
    
    if not success:
        print(" Training pipeline failed at model registration step")
        return False
    
    # Step 4: List Registered Models
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Registry Verification")
    
    success = run_python_script(
        script_path='src/deployment/model_registry.py',
        args=[
            '--config', 'config.yaml',
            '--action', 'list'
        ],
        description="List all registered model versions"
    )
    
    if not success:
        print(" Model listing failed, but registration may have succeeded")
    
    # Final status
    show_pipeline_status(pipeline_name, total_steps, total_steps, "Completed")
    
    print(f"\n {pipeline_name} completed successfully!")
    print(f"\n RESULTS:")
    print(f"   •  Model Version: {model_version}")
    print(f"   •  Artifacts saved in: models/")
    print(f"   •  Registered to SageMaker Model Registry")
    
    print("\n NEXT STEPS:")
    print("   1.  Deploy to development: Run deployment pipeline")
    print("   2.  Test endpoint functionality")
    print("   3.  Monitor model performance")
    print("   4.  Promote to staging when ready")
    
    return True

# Execute the training pipeline
print(" Ready to run model training pipeline...")
print(" This includes: training → evaluation → registration → verification")
print("\n Click 'Run' to start the pipeline or set run_training=False to skip")

# Set this to True to run the training pipeline
run_training = True

if run_training:
    training_success = run_training_pipeline()
    if training_success:
        print("\n Training pipeline execution completed successfully!")
    else:
        print("\n Training pipeline execution failed. Check logs for details.")
else:
    print(" Training pipeline execution skipped. Set run_training=True to execute.")

 Ready to run model training pipeline...
 This includes: training → evaluation → registration → verification

 Click 'Run' to start the pipeline or set run_training=False to skip

 STARTING: Model Training & Registry Pipeline
 Model Version: v20250623_153819

 Model Training & Registry Pipeline Progress: [] 25.0%
 Step 1/4: Model Training
------------------------------------------------------------

 EXECUTING: Python Script: train_model.py
 Description: Train forecasting models with evaluation
 Started at: 2025-06-23 15:38:19
------------------------------------------------------------
 Python Script: train_model.py completed successfully
 Execution time: 100.81 seconds
 Output:
2025-06-23 15:38:19,696 - ModelTrainer - INFO - ============================================================
2025-06-23 15:38:19,696 - ModelTrainer - INFO - LOGGING SETUP COMPLETED
2025-06-23 15:38:19,696 - ModelTrainer - INFO - ============================================================
2025-06-23 15:38:19,6

##  6. Deployment Pipeline - Multi-Environment

**Purpose**: Deploy trained models across different environments (dev, staging, production).

This pipeline performs:
1. **Development Deployment**: Deploy to dev environment for initial testing
2. **Staging Deployment**: Deploy to staging for integration testing  
3. **Production Deployment**: Deploy to production with monitoring
4. **Endpoint Testing**: Comprehensive functionality testing
5. **Health Monitoring**: Continuous endpoint health checks

**Features**:
- Multi-environment deployment strategy
- Automated endpoint testing
- Blue-green deployment support
- Rollback capabilities

**Equivalent Makefile Commands**: 
- `make deploy-dev`
- `make deploy-staging` 
- `make deploy-prod`
- `make workflow-staging`
- `make workflow-prod`

In [9]:
def deploy_to_environment(environment='dev', test_endpoint=True):
    """
    Deploy model to specified environment
    
    Args:
        environment: Target environment ('dev', 'staging', 'prod')
        test_endpoint: Whether to test the endpoint after deployment
    """
    pipeline_name = f"Deployment to {environment.upper()}"
    total_steps = 3 if test_endpoint else 2
    current_step = 0
    
    print(f"\n STARTING: {pipeline_name}")
    print("=" * 70)
    
    # Configuration
    model_name = "chinese_produce_forecaster"
    endpoint_name = f"produce-forecast-{environment}"
    model_path = "models/best_model.pkl"
    
    print(f" Deployment Configuration:")
    print(f"   •  Environment: {environment}")
    print(f"   •  Model: {model_name}")
    print(f"   •  Endpoint: {endpoint_name}")
    print(f"   •  Model Path: {model_path}")
    
    # Step 1: Model Deployment
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Model Deployment")
    
    success = run_python_script(
        script_path='src/deployment/sagemaker_deploy.py',
        args=[
            '--config', 'config.yaml',
            '--action', 'deploy',
            '--model-path', model_path,
            '--model-name', model_name,
            '--endpoint-name', endpoint_name,
            '--environment', environment
        ],
        description=f"Deploy model to {environment} environment"
    )
    
    if not success:
        print(f" {pipeline_name} failed at deployment step")
        return False
    
    # Step 2: Verify Endpoint Status
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Endpoint Verification")
    
    try:
        sagemaker_client = boto3.client('sagemaker', region_name=config['aws']['region'])
        
        # Check endpoint status
        response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        endpoint_status = response['EndpointStatus']
        
        print(f"🔍 Endpoint Status: {endpoint_status}")
        
        if endpoint_status == 'InService':
            print(f" Endpoint '{endpoint_name}' is ready for inference")
            creation_time = response['CreationTime']
            print(f" Created: {creation_time}")
        else:
            print(f" Endpoint status: {endpoint_status}")
            
    except Exception as e:
        print(f" Could not verify endpoint status: {e}")
        return False
    
    # Step 3: Test Endpoint (if requested)
    if test_endpoint:
        current_step += 1
        show_pipeline_status(pipeline_name, current_step, total_steps, "Endpoint Testing")
        
        success = run_python_script(
            script_path='src/deployment/sagemaker_deploy.py',
            args=[
                '--config', 'config.yaml',
                '--action', 'test',
                '--endpoint-name', endpoint_name
            ],
            description="Test endpoint functionality with sample data"
        )
        
        if not success:
            print(" Endpoint testing failed, but deployment succeeded")
    
    # Final status
    show_pipeline_status(pipeline_name, total_steps, total_steps, "Completed")
    
    print(f"\n {pipeline_name} completed successfully!")
    print(f"\n DEPLOYMENT SUMMARY:")
    print(f"   •  Environment: {environment}")
    print(f"   •  Endpoint: {endpoint_name}")
    print(f"   •  Status: Ready for inference")
    
    if environment == 'dev':
        print("\n NEXT STEPS:")
        print("   1.  Run integration tests")
        print("   2.  Monitor performance metrics")
        print("   3.  Promote to staging when ready")
    elif environment == 'staging':
        print("\n NEXT STEPS:")
        print("   1.  Run full test suite")
        print("   2.  User acceptance testing")
        print("   3.  Promote to production when approved")
    elif environment == 'prod':
        print("\n NEXT STEPS:")
        print("   1.  Start monitoring pipeline")
        print("   2.  Configure alerts")
        print("   3.  Monitor business metrics")
    
    return True

def run_complete_deployment_workflow():
    """
    Run complete deployment workflow: dev → staging → prod
    """
    print("\n COMPLETE DEPLOYMENT WORKFLOW")
    print("=" * 70)
    print(" This will deploy to: dev → staging → production")
    print(" This process may take 15-30 minutes")
    
    # Deploy to development
    print("\n Phase 1: Development Deployment")
    dev_success = deploy_to_environment('dev', test_endpoint=True)
    
    if not dev_success:
        print(" Development deployment failed. Stopping workflow.")
        return False
    
    # Deploy to staging
    print("\n Phase 2: Staging Deployment")
    staging_success = deploy_to_environment('staging', test_endpoint=True)
    
    if not staging_success:
        print(" Staging deployment failed. Development is still available.")
        return False
    
    # Deploy to production (with monitoring)
    print("\n Phase 3: Production Deployment")
    prod_success = deploy_to_environment('prod', test_endpoint=True)
    
    if not prod_success:
        print(" Production deployment failed. Staging is still available.")
        return False
    
    print("\n COMPLETE DEPLOYMENT WORKFLOW SUCCESSFUL!")
    print("\n ACTIVE ENDPOINTS:")
    print("   •  Development: produce-forecast-dev")
    print("   •  Staging: produce-forecast-staging")
    print("   •  Production: produce-forecast-prod")
    
    return True

# Deployment Options
print(" Ready to run deployment pipeline...")
print("\n DEPLOYMENT OPTIONS:")
print("    Single environment deployment")
print("    Complete workflow (dev → staging → prod)")
print("\n Configure your deployment below:")

# Configuration
deployment_type = "single"  # Options: "single" or "complete"
target_environment = "dev"  # Options: "dev", "staging", "prod"
run_deployment = True      # Set to True to run deployment

if run_deployment:
    if deployment_type == "single":
        deployment_success = deploy_to_environment(target_environment, test_endpoint=True)
    elif deployment_type == "complete":
        deployment_success = run_complete_deployment_workflow()
    else:
        print(" Invalid deployment_type. Use 'single' or 'complete'")
        deployment_success = False
    
    if deployment_success:
        print("\n Deployment pipeline execution completed successfully!")
    else:
        print("\n Deployment pipeline execution failed. Check logs for details.")
else:
    print(" Deployment pipeline execution skipped. Set run_deployment=True to execute.")

 Ready to run deployment pipeline...

 DEPLOYMENT OPTIONS:
    Single environment deployment
    Complete workflow (dev → staging → prod)

 Configure your deployment below:

 STARTING: Deployment to DEV
 Deployment Configuration:
   •  Environment: dev
   •  Model: chinese_produce_forecaster
   •  Endpoint: produce-forecast-dev
   •  Model Path: models/best_model.pkl

 Deployment to DEV Progress: [] 33.3%
 Step 1/3: Model Deployment
------------------------------------------------------------

 EXECUTING: Python Script: sagemaker_deploy.py
 Description: Deploy model to dev environment
 Started at: 2025-06-23 15:42:25
------------------------------------------------------------
 Python Script: sagemaker_deploy.py completed successfully
 Execution time: 244.93 seconds
 Output:
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
--

##  7. Monitoring & Observability Pipeline

**Purpose**: Start comprehensive monitoring systems for deployed models.

This pipeline performs:
1. **Performance Monitoring**: Real-time model performance tracking
2. **Data Drift Detection**: Monitor input data distribution changes
3. **Dashboard Setup**: Interactive monitoring dashboards
4. **Alert Configuration**: Automated alerting for anomalies
5. **Health Checks**: Continuous system health monitoring

**Features**:
- Real-time metrics collection
- Interactive Plotly/Dash dashboards  
- Statistical drift detection algorithms
- Configurable alerting thresholds
- Historical performance analysis

**Equivalent Makefile Commands**: 
- `make monitoring-start`
- `make monitoring-status`
- `make detect-drift`
- `make performance-report`

In [11]:
import subprocess
import psutil
import signal
import socket
from contextlib import closing

def find_free_port(start_port=8050, max_attempts=10):
    """Find a free port starting from start_port"""
    for i in range(max_attempts):
        port = start_port + i
        with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as sock:
            if sock.connect_ex(('localhost', port)) != 0:
                return port
    return None

def stop_monitoring_processes():
    """Stop all existing monitoring processes"""
    print(" Stopping existing monitoring processes...")
    
    # Kill processes by port
    ports_to_check = [8002, 8050, 8051, 8052]
    for port in ports_to_check:
        try:
            for proc in psutil.process_iter(['pid', 'name', 'connections']):
                if proc.info['connections']:
                    for conn in proc.info['connections']:
                        if hasattr(conn, 'laddr') and conn.laddr.port == port:
                            print(f"    Killing process on port {port} (PID: {proc.pid})")
                            proc.terminate()
                            break
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            continue
    
    # Kill processes by name pattern
    patterns = ['performance_monitor.py', 'drift_detector.py']
    for pattern in patterns:
        try:
            subprocess.run(['pkill', '-f', pattern], 
                         capture_output=True, timeout=5)
        except (subprocess.TimeoutExpired, subprocess.CalledProcessError):
            pass
    
    time.sleep(2)
    print(" Monitoring cleanup completed")

def start_monitoring_component(component_name, script_path, args, background=True):
    """Start a monitoring component"""
    command = ['python3', str(script_path)] + args
    
    try:
        if background:
            process = subprocess.Popen(
                command,
                cwd=PROJECT_ROOT,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            print(f" {component_name} started (PID: {process.pid})")
            return process
        else:
            result = subprocess.run(
                command,
                cwd=PROJECT_ROOT,
                capture_output=True,
                text=True,
                timeout=30
            )
            if result.returncode == 0:
                print(f" {component_name} executed successfully")
                if result.stdout.strip():
                    print(f"📤 Output: {result.stdout}")
                return True
            else:
                print(f" {component_name} failed: {result.stderr}")
                return False
                
    except Exception as e:
        print(f" Failed to start {component_name}: {e}")
        return None

def run_monitoring_pipeline(start_services=True, run_drift_analysis=True):
    """
    Execute the complete monitoring pipeline
    
    Args:
        start_services: Whether to start background monitoring services
        run_drift_analysis: Whether to run drift detection analysis
    """
    pipeline_name = "Monitoring & Observability Pipeline"
    total_steps = 5 if start_services else 3
    current_step = 0
    
    print(f"\n STARTING: {pipeline_name}")
    print("=" * 70)
    
    # Configuration
    monitoring_interval = 60
    reference_data = "data/processed/train.parquet"
    current_data = "data/processed/validation.parquet"
    
    print(f" Monitoring Configuration:")
    print(f"   •  Interval: {monitoring_interval} seconds")
    print(f"   •  Reference Data: {reference_data}")
    print(f"   •  Current Data: {current_data}")
    
    # Step 1: Stop existing monitoring
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Cleanup")
    stop_monitoring_processes()
    
    # Step 2: Performance Health Check
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Performance Check")
    
    success = start_monitoring_component(
        component_name="Performance Health Check",
        script_path='src/monitoring/performance_monitor.py',
        args=[
            '--config', 'config.yaml',
            '--action', 'health',
            '--local-mode'
        ],
        background=False
    )
    
    if not success:
        print(" Performance health check failed, continuing...")
    
    # Step 3: Drift Detection Analysis
    if run_drift_analysis:
        current_step += 1
        show_pipeline_status(pipeline_name, current_step, total_steps, "Drift Analysis")
        
        success = start_monitoring_component(
            component_name="Drift Detection Analysis",
            script_path='src/monitoring/drift_detector.py',
            args=[
                '--config', 'config.yaml',
                '--action', 'detect',
                '--reference-data', reference_data,
                '--current-data', current_data,
                '--local-mode',
                '--generate-report'
            ],
            background=False
        )
        
        if not success:
            print(" Drift analysis failed, continuing...")
    
    if start_services:
        # Step 4: Start Performance Monitor
        current_step += 1
        show_pipeline_status(pipeline_name, current_step, total_steps, "Performance Monitor")
        
        perf_process = start_monitoring_component(
            component_name="Performance Monitor",
            script_path='src/monitoring/performance_monitor.py',
            args=[
                '--config', 'config.yaml',
                '--action', 'start',
                '--interval', str(monitoring_interval),
                '--local-mode'
            ],
            background=True
        )
        
        time.sleep(2)  # Let performance monitor start
        
        # Step 5: Start Drift Monitor
        current_step += 1
        show_pipeline_status(pipeline_name, current_step, total_steps, "Drift Monitor")
        
        drift_process = start_monitoring_component(
            component_name="Drift Monitor",
            script_path='src/monitoring/drift_detector.py',
            args=[
                '--config', 'config.yaml',
                '--action', 'monitor',
                '--reference-data', reference_data,
                '--current-data', current_data,
                '--local-mode',
                '--interval', '60'
            ],
            background=True
        )
        
        time.sleep(3)  # Let drift monitor start
        
        # Try to start dashboard on available port
        dashboard_port = find_free_port(8050)
        if dashboard_port:
            print(f"\n Starting monitoring dashboard on port {dashboard_port}...")
            dashboard_process = start_monitoring_component(
                component_name="Monitoring Dashboard",
                script_path='src/monitoring/performance_monitor.py',
                args=[
                    '--config', 'config.yaml',
                    '--action', 'dashboard',
                    '--port', str(dashboard_port),
                    '--local-mode'
                ],
                background=True
            )
            
            if dashboard_process:
                print(f" Dashboard will be available at: http://localhost:{dashboard_port}")
        
        time.sleep(3)  # Let everything stabilize
    
    # Final status
    show_pipeline_status(pipeline_name, total_steps, total_steps, "Completed")
    
    print(f"\n {pipeline_name} completed successfully!")
    
    if start_services:
        print(f"\n ACTIVE MONITORING SERVICES:")
        print(f"   •  Performance Monitor: Running")
        print(f"   •  Drift Detection: Running")
        if dashboard_port:
            print(f"   •  Dashboard: http://localhost:{dashboard_port}")
        
        print("\n MONITORING COMMANDS:")
        print("   •  Check status: Run monitoring status check below")
        print("   •  Stop services: Run monitoring cleanup below")
        print("   •  View logs: Check data/monitoring/logs/")
    
    print("\n NEXT STEPS:")
    print("   1.  Monitor dashboard for real-time metrics")
    print("   2.  Configure alert thresholds")
    print("   3.  Review performance trends")
    print("   4.  Set up automated reporting")
    
    return True

def check_monitoring_status():
    """Check status of all monitoring components"""
    print("\n MONITORING SYSTEM STATUS")
    print("=" * 50)
    
    # Check running processes
    monitoring_processes = []
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        try:
            cmdline = ' '.join(proc.info['cmdline']) if proc.info['cmdline'] else ''
            if any(pattern in cmdline for pattern in ['performance_monitor.py', 'drift_detector.py']):
                monitoring_processes.append({
                    'pid': proc.info['pid'],
                    'name': proc.info['name'],
                    'cmdline': cmdline
                })
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            continue
    
    if monitoring_processes:
        print(f" Found {len(monitoring_processes)} monitoring processes:")
        for proc in monitoring_processes:
            process_type = "Performance" if "performance_monitor" in proc['cmdline'] else "Drift Detection"
            print(f"   • {process_type}: PID {proc['pid']}")
    else:
        print(" No monitoring processes found")
    
    # Check port usage
    ports_to_check = [8002, 8050, 8051, 8052]
    print("\n Port Status:")
    for port in ports_to_check:
        with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as sock:
            if sock.connect_ex(('localhost', port)) == 0:
                print(f"   • Port {port}:  In Use")
            else:
                print(f"   • Port {port}: ⚪ Available")
    
    return len(monitoring_processes) > 0

# Execute monitoring pipeline
print(" Ready to run monitoring pipeline...")
print(" This includes: performance monitoring + drift detection + dashboards")
print("\n Configure your monitoring below:")

# Configuration
start_background_services = True  # Start continuous monitoring services
run_analysis_only = False        # Run one-time analysis instead
run_monitoring = True            # Set to True to execute

if run_monitoring:
    if run_analysis_only:
        monitoring_success = run_monitoring_pipeline(start_services=False, run_drift_analysis=True)
    else:
        monitoring_success = run_monitoring_pipeline(start_services=start_background_services, run_drift_analysis=True)
    
    if monitoring_success:
        print("\n Monitoring pipeline execution completed successfully!")
    else:
        print("\n Monitoring pipeline execution failed. Check logs for details.")
else:
    print(" Monitoring pipeline execution skipped. Set run_monitoring=True to execute.")

 Ready to run monitoring pipeline...
 This includes: performance monitoring + drift detection + dashboards

 Configure your monitoring below:

 STARTING: Monitoring & Observability Pipeline
 Monitoring Configuration:
   •  Interval: 60 seconds
   •  Reference Data: data/processed/train.parquet
   •  Current Data: data/processed/validation.parquet

 Monitoring & Observability Pipeline Progress: [] 20.0%
 Step 1/5: Cleanup
------------------------------------------------------------
 Stopping existing monitoring processes...
    Killing process on port 8050 (PID: 25420)
 Monitoring cleanup completed

 Monitoring & Observability Pipeline Progress: [] 40.0%
 Step 2/5: Performance Check
------------------------------------------------------------
 Performance Health Check executed successfully
📤 Output: Collecting current metrics for health report...

System Health Summary
Overall Status: WARNING
Timestamp: 2025-06-23T15:51:17.094926
Local Mode: True
AWS Enabled: False

System Health: HEALT

## 8. Monitoring Status & Control Panel

**Purpose**: Monitor and control all running services and processes.

This section provides:
- Real-time status of all monitoring services
- Port usage and availability checking
- Service control (start/stop/restart)
- Log file access and viewing
- Emergency shutdown capabilities

In [12]:
# Check current monitoring status
print(" MONITORING STATUS CHECK")
print("=" * 50)

monitoring_active = check_monitoring_status()

if monitoring_active:
    print("\n Monitoring services are running")
else:
    print("\n No monitoring services detected")
    print("💡 Run the monitoring pipeline above to start services")

# Service control options
print("\n SERVICE CONTROL OPTIONS:")
print("   1.  Status Check: Already completed above")
print("   2.  Stop Services: Run the cell below")
print("   3.  Restart Services: Stop then run monitoring pipeline")
print("   4.  View Logs: Check data/monitoring/logs/ directory")

 MONITORING STATUS CHECK

 MONITORING SYSTEM STATUS
 Found 3 monitoring processes:
   • Performance: PID 25539
   • Drift Detection: PID 25542
   • Performance: PID 25610

 Port Status:
   • Port 8002: ⚪ Available
   • Port 8050:  In Use
   • Port 8051: ⚪ Available
   • Port 8052: ⚪ Available

 Monitoring services are running

 SERVICE CONTROL OPTIONS:
   1.  Status Check: Already completed above
   2.  Stop Services: Run the cell below
   3.  Restart Services: Stop then run monitoring pipeline
   4.  View Logs: Check data/monitoring/logs/ directory


In [13]:
# EMERGENCY STOP - Run this cell to stop all monitoring services
stop_all_services = False  # Set to True to stop all services

if stop_all_services:
    print(" EMERGENCY STOP - Stopping all monitoring services...")
    stop_monitoring_processes()
    
    # Verify shutdown
    time.sleep(2)
    remaining_processes = check_monitoring_status()
    
    if not remaining_processes:
        print(" All monitoring services stopped successfully")
    else:
        print(" Some processes may still be running")
else:
    print(" Service stop skipped. Set stop_all_services=True to stop services.")

 Service stop skipped. Set stop_all_services=True to stop services.


##  9. Testing & Quality Assurance Pipeline

**Purpose**: Run comprehensive testing and quality checks on the codebase.

This pipeline performs:
1. **Unit Tests**: Test individual components and functions
2. **Integration Tests**: Test component interactions
3. **Code Quality**: Linting, formatting, and style checks
4. **Security Scanning**: Vulnerability detection and dependency checks
5. **Coverage Analysis**: Test coverage reporting

**Equivalent Makefile Commands**: 
- `make test-full`
- `make quality-check`
- `make security-check`
- `make test-coverage`

In [15]:
def run_testing_pipeline(include_security=True, include_coverage=True):
    """
    Execute comprehensive testing and quality assurance pipeline
    
    Args:
        include_security: Whether to run security scans
        include_coverage: Whether to generate coverage reports
    """
    pipeline_name = "Testing & Quality Assurance Pipeline"
    total_steps = 5 if include_security and include_coverage else 3
    current_step = 0
    
    print(f"\n STARTING: {pipeline_name}")
    print("=" * 70)
    
    # Step 1: Code Formatting
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Code Formatting")
    
    # Black formatting
    success = run_pipeline_step(
        step_name="Black Code Formatting",
        command=['black', 'src/', 'tests/', '--line-length=100', '--check'],
        description="Check code formatting with Black"
    )
    
    if not success:
        print(" Code formatting issues found. Running auto-format...")
        run_pipeline_step(
            step_name="Auto-format with Black",
            command=['black', 'src/', 'tests/', '--line-length=100'],
            description="Auto-format code with Black"
        )
    
    # Import sorting
    run_pipeline_step(
        step_name="Import Sorting",
        command=['isort', 'src/', 'tests/', '--profile', 'black', '--check-only'],
        description="Check import sorting with isort"
    )
    
    # Step 2: Code Linting
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Code Linting")
    
    # Flake8 linting
    success = run_pipeline_step(
        step_name="Flake8 Linting",
        command=['flake8', 'src/', '--max-line-length=100', '--ignore=E203,W503'],
        description="Check code quality with Flake8"
    )
    
    # MyPy type checking (optional - may not be configured)
    mypy_success = run_pipeline_step(
        step_name="MyPy Type Checking",
        command=['mypy', 'src/', '--ignore-missing-imports'],
        description="Static type checking with MyPy"
    )
    
    if not mypy_success:
        print(" MyPy type checking found issues (non-blocking)")
    
    # Step 3: Unit and Integration Tests
    current_step += 1
    show_pipeline_status(pipeline_name, current_step, total_steps, "Test Execution")
    
    if include_coverage:
        # Run tests with coverage
        success = run_pipeline_step(
            step_name="Pytest with Coverage",
            command=[
                'python3', '-m', 'pytest', 'tests/',
                '--cov=src/',
                '--cov-report=html',
                '--cov-report=term-missing',
                '--cov-report=xml',
                '-v'
            ],
            description="Run tests with coverage analysis",
            check_files=['htmlcov/index.html', 'coverage.xml']
        )
    else:
        # Run tests without coverage
        success = run_pipeline_step(
            step_name="Pytest",
            command=['python3', '-m', 'pytest', 'tests/', '-v'],
            description="Run unit and integration tests"
        )
    
    if not success:
        print(" Test execution failed. Check test output for details.")
        return False
    
    if include_security:
        # Step 4: Security Scanning
        current_step += 1
        show_pipeline_status(pipeline_name, current_step, total_steps, "Security Scanning")
        
        # Bandit security scan
        bandit_success = run_pipeline_step(
            step_name="Bandit Security Scan",
            command=[
                'bandit', '-r', 'src/',
                '-f', 'json',
                '-o', 'security_report.json'
            ],
            description="Security vulnerability scanning with Bandit",
            check_files=['security_report.json']
        )
        
        # Safety dependency check
        safety_success = run_pipeline_step(
            step_name="Safety Dependency Check",
            command=['safety', 'check'],
            description="Check dependencies for known vulnerabilities"
        )
        
        if not bandit_success or not safety_success:
            print(" Security issues detected. Review security_report.json")
    
    # Final status
    current_step = total_steps
    show_pipeline_status(pipeline_name, current_step, total_steps, "Completed")
    
    print(f"\n {pipeline_name} completed!")
    
    # Summary
    print(f"\n QUALITY ASSURANCE SUMMARY:")
    print(f"   •  Code formatting checked")
    print(f"   •  Linting completed")
    print(f"   •  Tests executed")
    
    if include_coverage:
        print(f"   •  Coverage report: htmlcov/index.html")
    
    if include_security:
        print(f"   •  Security scan completed")
        print(f"   •  Security report: security_report.json")
    
    print("\n NEXT STEPS:")
    print("   1.  Review test coverage report")
    print("   2.  Address any security findings")
    print("   3.  Ready for deployment")
    
    return True

# Execute testing pipeline
print(" Ready to run testing and quality assurance pipeline...")
print("💡 This includes: formatting + linting + testing + security + coverage")
print("\n⚡ Configure testing options below:")

# Configuration
run_security_scan = True
generate_coverage = True
run_testing = True

if run_testing:
    testing_success = run_testing_pipeline(
        include_security=run_security_scan,
        include_coverage=generate_coverage
    )
    
    if testing_success:
        print("\n Testing pipeline execution completed successfully!")
    else:
        print("\n Testing pipeline execution failed. Check logs for details.")
else:
    print(" Testing pipeline execution skipped. Set run_testing=True to execute.")

 Ready to run testing and quality assurance pipeline...
💡 This includes: formatting + linting + testing + security + coverage

⚡ Configure testing options below:

 STARTING: Testing & Quality Assurance Pipeline

 Testing & Quality Assurance Pipeline Progress: [] 20.0%
 Step 1/5: Code Formatting
------------------------------------------------------------

 EXECUTING: Black Code Formatting
 Description: Check code formatting with Black
 Started at: 2025-06-23 15:55:57
------------------------------------------------------------
 Black Code Formatting failed with exception: [Errno 2] No such file or directory: 'black'
 Execution time: 0.00 seconds
 Code formatting issues found. Running auto-format...

 EXECUTING: Auto-format with Black
 Description: Auto-format code with Black
 Started at: 2025-06-23 15:55:57
------------------------------------------------------------
 Auto-format with Black failed with exception: [Errno 2] No such file or directory: 'black'
 Execution time: 0.00 second

##  10. Master Workflow Orchestrator

**Purpose**: Execute complete end-to-end MLOps workflows with a single command.

This section provides pre-configured workflows for different scenarios:

###  Available Workflows:

1. **Development Workflow**: Data pipeline → Training → Testing → Dev deployment
2. **Staging Workflow**: Complete pipeline → Staging deployment → Monitoring
3. **Production Workflow**: Full validation → Production deployment → Full monitoring
4. **CI/CD Simulation**: Testing → Quality checks → Deployment pipeline
5. **Complete MLOps**: Everything - data to production with monitoring

**Equivalent Makefile Commands**: 
- `make workflow-dev`
- `make workflow-staging`
- `make workflow-prod`
- `make pipeline-full`
- `make pipeline-production`

In [16]:
def run_development_workflow():
    """
    Complete development workflow: data → training → testing → dev deployment
    """
    workflow_name = "Development Workflow"
    print(f"\n STARTING: {workflow_name}")
    print("=" * 70)
    print(" Pipeline: Data → Training → Testing → Dev Deployment")
    
    # Phase 1: Data Pipeline
    print("\n Phase 1: Data Pipeline")
    if not run_data_pipeline_full(include_bi=False):
        print(" Development workflow failed at data pipeline")
        return False
    
    # Phase 2: Model Training
    print("\n Phase 2: Model Training")
    if not run_training_pipeline():
        print(" Development workflow failed at training")
        return False
    
    # Phase 3: Fast Testing
    print("\n Phase 3: Fast Testing")
    if not run_testing_pipeline(include_security=False, include_coverage=False):
        print(" Development workflow failed at testing")
        return False
    
    # Phase 4: Dev Deployment
    print("\n Phase 4: Development Deployment")
    if not deploy_to_environment('dev', test_endpoint=True):
        print(" Development workflow failed at deployment")
        return False
    
    print(f"\n {workflow_name} completed successfully!")
    return True

def run_staging_workflow():
    """
    Complete staging workflow: full pipeline → staging deployment → monitoring
    """
    workflow_name = "Staging Workflow"
    print(f"\n STARTING: {workflow_name}")
    print("=" * 70)
    print(" Pipeline: Full Data + Training → Staging → Monitoring")
    
    # Phase 1: Complete Data + Training Pipeline
    print("\n Phase 1: Complete Data Pipeline")
    if not run_data_pipeline_full(include_bi=True):
        print(" Staging workflow failed at data pipeline")
        return False
    
    print("\n Phase 2: Model Training & Registry")
    if not run_training_pipeline():
        print(" Staging workflow failed at training")
        return False
    
    # Phase 2: Staging Deployment
    print("\n Phase 3: Staging Deployment")
    if not deploy_to_environment('staging', test_endpoint=True):
        print(" Staging workflow failed at deployment")
        return False
    
    # Phase 3: Start Monitoring
    print("\n Phase 4: Monitoring Setup")
    if not run_monitoring_pipeline(start_services=True, run_drift_analysis=True):
        print(" Monitoring setup failed, but deployment succeeded")
    
    print(f"\n {workflow_name} completed successfully!")
    return True

def run_production_workflow():
    """
    Complete production workflow: full validation → production → comprehensive monitoring
    """
    workflow_name = "Production Workflow"
    print(f"\n STARTING: {workflow_name}")
    print("=" * 70)
    print(" Pipeline: Full Validation → Production → Comprehensive Monitoring")
    
    # Phase 1: Full Testing & Quality
    print("\n Phase 1: Comprehensive Testing")
    if not run_testing_pipeline(include_security=True, include_coverage=True):
        print(" Production workflow failed at testing")
        return False
    
    # Phase 2: Production Deployment
    print("\n Phase 2: Production Deployment")
    if not deploy_to_environment('prod', test_endpoint=True):
        print(" Production workflow failed at deployment")
        return False
    
    # Phase 3: Comprehensive Monitoring
    print("\n Phase 3: Production Monitoring")
    if not run_monitoring_pipeline(start_services=True, run_drift_analysis=True):
        print(" Monitoring setup failed, but deployment succeeded")
    
    print(f"\n {workflow_name} completed successfully!")
    print("\n Your model is now live in production with full monitoring!")
    return True

def run_complete_mlops_pipeline():
    """
    Complete end-to-end MLOps pipeline: everything from data to production
    """
    workflow_name = "Complete MLOps Pipeline"
    print(f"\n STARTING: {workflow_name}")
    print("=" * 70)
    print(" End-to-End: Data → Training → Testing → Multi-Env Deployment → Monitoring")
    print(" This will take 30-60 minutes to complete")
    
    start_time = time.time()
    
    # Phase 1: Complete Data Pipeline
    print("\n Phase 1: Complete Data Pipeline")
    if not run_data_pipeline_full(include_bi=True):
        print(" Complete MLOps pipeline failed at data phase")
        return False
    
    # Phase 2: Model Training & Registry
    print("\n Phase 2: Model Training & Registry")
    if not run_training_pipeline():
        print(" Complete MLOps pipeline failed at training phase")
        return False
    
    # Phase 3: Quality Assurance
    print("\n Phase 3: Quality Assurance")
    if not run_testing_pipeline(include_security=True, include_coverage=True):
        print(" Complete MLOps pipeline failed at testing phase")
        return False
    
    # Phase 4: Multi-Environment Deployment
    print("\n Phase 4: Multi-Environment Deployment")
    if not run_complete_deployment_workflow():
        print(" Complete MLOps pipeline failed at deployment phase")
        return False
    
    # Phase 5: Production Monitoring
    print("\n Phase 5: Production Monitoring")
    if not run_monitoring_pipeline(start_services=True, run_drift_analysis=True):
        print(" Monitoring setup failed, but deployment succeeded")
    
    # Final summary
    execution_time = (time.time() - start_time) / 60
    
    print(f"\n {workflow_name} completed successfully!")
    print(f" Total execution time: {execution_time:.1f} minutes")
    
    print("\n DEPLOYMENT SUMMARY:")
    print("   •  Data pipeline with Feature Store and Athena")
    print("   •  Trained models registered in Model Registry")
    print("   •  Full test suite with security and coverage")
    print("   •  Multi-environment deployment (dev/staging/prod)")
    print("   •  Real-time monitoring and drift detection")
    
    print("\n Your complete MLOps pipeline is now operational!")
    return True

# Workflow Selection Interface
print(" MASTER WORKFLOW ORCHESTRATOR")
print("=" * 50)
print("\n Available Workflows:")
print("   1.  Development: Fast iteration workflow")
print("   2.  Staging: Complete pipeline with staging deployment")
print("   3.  Production: Full validation with production deployment")
print("   4.  Complete MLOps: End-to-end everything (30-60 min)")

# Configuration - Choose your workflow
selected_workflow = "development"  # Options: "development", "staging", "production", "complete"
run_workflow = True               # Set to True to execute

print(f"\n⚡ Selected workflow: {selected_workflow}")
print(f" Execution: {'ENABLED' if run_workflow else 'DISABLED'}")

if run_workflow:
    workflow_success = False
    
    if selected_workflow == "development":
        workflow_success = run_development_workflow()
    elif selected_workflow == "staging":
        workflow_success = run_staging_workflow()
    elif selected_workflow == "production":
        workflow_success = run_production_workflow()
    elif selected_workflow == "complete":
        workflow_success = run_complete_mlops_pipeline()
    else:
        print(f" Unknown workflow: {selected_workflow}")
        print("Available options: development, staging, production, complete")
    
    if workflow_success:
        print("\n Master workflow execution completed successfully!")
        print(" Your MLOps pipeline is ready!")
    else:
        print("\n Master workflow execution failed. Check logs for details.")
else:
    print("\n Workflow execution skipped. Set run_workflow=True to execute.")
    print(" Choose your workflow by setting selected_workflow variable")

 MASTER WORKFLOW ORCHESTRATOR

 Available Workflows:
   1.  Development: Fast iteration workflow
   2.  Staging: Complete pipeline with staging deployment
   3.  Production: Full validation with production deployment
   4.  Complete MLOps: End-to-end everything (30-60 min)

⚡ Selected workflow: development
 Execution: ENABLED

 STARTING: Development Workflow
 Pipeline: Data → Training → Testing → Dev Deployment

 Phase 1: Data Pipeline

 STARTING: Complete Data Pipeline
 Creating project directories...
    data/raw
    data/processed
    data/validation
    data/monitoring/logs
    data/monitoring/reports
    data/monitoring/metrics
    models
    reports
 All directories created successfully!

 Complete Data Pipeline Progress: [] 25.0%
 Step 1/4: Data Validation
------------------------------------------------------------

 EXECUTING: Python Script: data_validation.py
 Description: Validate data quality, schema, and consistency
 Started at: 2025-06-23 15:59:14
------------------------

##  11. Project Summary & Next Steps

**Purpose**: Provide a comprehensive summary of the MLOps pipeline execution and guide next steps.

This final section summarizes:
-  Completed pipeline components
-  Active endpoints and services  
-  Available dashboards and monitoring
-  Recommended next steps
-  Troubleshooting guidance

---

In [19]:
def generate_project_summary():
    """
    Generate comprehensive project summary and status
    """
    print("\n MLOPS PROJECT SUMMARY")
    print("=" * 70)
    
    # Project Information
    print(f"\n PROJECT INFORMATION:")
    print(f"   •  Project: Demand Stock Forecasting MLOps")
    print(f"   •  Author: Bhupal Lambodhar")
    print(f"   •  Email: btiduwarlambodhar@sandiego.edu")
    print(f"   •  Repository: https://github.com/btlambodh/demand-stock-forecasting-mlops")
    print(f"   •  Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Check system status
    print(f"\n SYSTEM STATUS:")
    
    # Check data files
    data_path = PROJECT_ROOT / 'data'
    models_path = PROJECT_ROOT / 'models'
    
    if data_path.exists():
        data_files = list(data_path.rglob('*.csv')) + list(data_path.rglob('*.parquet'))
        print(f"   •  Data Files: {len(data_files)} found")
    else:
        print(f"   •  Data Files:  No data directory")
    
    if models_path.exists():
        model_files = list(models_path.rglob('*.pkl')) + list(models_path.rglob('*.joblib'))
        print(f"   •  Model Files: {len(model_files)} found")
        
        # Check for best model
        best_model = models_path / 'best_model.pkl'
        if best_model.exists():
            print(f"   •  Best Model:  Available")
        else:
            print(f"   •  Best Model:  Not found")
    else:
        print(f"   •  Model Files:  No models directory")
    
    # Check SageMaker endpoints
    try:
        sagemaker_client = boto3.client('sagemaker', region_name=config['aws']['region'])
        endpoints = sagemaker_client.list_endpoints()['Endpoints']
        
        active_endpoints = [ep for ep in endpoints if ep['EndpointStatus'] == 'InService']
        print(f"   •  SageMaker Endpoints: {len(active_endpoints)} active")
        
        for endpoint in active_endpoints:
            print(f"     - {endpoint['EndpointName']}: {endpoint['EndpointStatus']}")
            
    except Exception as e:
        print(f"   •  SageMaker Endpoints:  Cannot check ({e})")
    
    # Check monitoring status
    print(f"\n MONITORING STATUS:")
    monitoring_active = check_monitoring_status()
    
    if monitoring_active:
        print(f"   •  Monitoring Services:  Active")
    else:
        print(f"   •  Monitoring Services:  Not running")
    
    # Check available dashboards
    ports_to_check = [8050, 8051, 8052]
    active_dashboards = []
    
    for port in ports_to_check:
        with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as sock:
            if sock.connect_ex(('localhost', port)) == 0:
                active_dashboards.append(f"http://localhost:{port}")
    
    if active_dashboards:
        print(f"   •  Active Dashboards:")
        for dashboard in active_dashboards:
            print(f"     - {dashboard}")
    else:
        print(f"   •  Dashboards:  None active")
    
    # Next steps recommendations
    print(f"\n RECOMMENDED NEXT STEPS:")
    
    if not data_path.exists() or len(list(data_path.rglob('*.csv'))) == 0:
        print(f"   1.  Add training data to data/raw/ directory")
        print(f"   2.  Run data pipeline to process features")
    elif not models_path.exists() or not (models_path / 'best_model.pkl').exists():
        print(f"   1.  Run training pipeline to create models")
        print(f"   2.  Validate model performance")
    else:
        print(f"   1.  Deploy models to different environments")
        print(f"   2.  Monitor model performance in production")
        print(f"   3.  Set up automated retraining")
        print(f"   4.  Analyze business impact")
    
    # Troubleshooting guide
    print(f"\n TROUBLESHOOTING GUIDE:")
    print(f"   •  Data Issues: Check data/raw/ for input files")
    print(f"   •  Model Issues: Review models/ directory and logs")
    print(f"   •  AWS Issues: Verify credentials and permissions")
    print(f"   •  Monitoring Issues: Check port availability and processes")
    print(f"   •  Test Failures: Review test output and fix issues")
    
    # Useful commands
    print(f"\n USEFUL COMMANDS:")
    print(f"   • Status check: Run health check cells above")
    print(f"   • Stop services: Set stop_all_services=True and run")
    print(f"   • View logs: Check data/monitoring/logs/ directory")
    print(f"   • AWS Console: Check SageMaker endpoints and Model Registry")
    
    print(f"\n MLOps Pipeline Report Complete!")
    print(f" Support: btiduwarlambodhar@sandiego.edu")

# Generate the summary
generate_project_summary()


 MLOPS PROJECT SUMMARY

 PROJECT INFORMATION:
   •  Project: Demand Stock Forecasting MLOps
   •  Author: Bhupal Lambodhar
   •  Email: btiduwarlambodhar@sandiego.edu
   •  Repository: https://github.com/btlambodh/demand-stock-forecasting-mlops
   •  Report Generated: 2025-06-23 16:09:54

 SYSTEM STATUS:
   •  Data Files: 8 found
   •  Model Files: 6 found
   •  Best Model:  Available
   •  SageMaker Endpoints: 1 active
     - produce-forecast-dev: InService

 MONITORING STATUS:

 MONITORING SYSTEM STATUS
 Found 3 monitoring processes:
   • Performance: PID 25539
   • Drift Detection: PID 25542
   • Performance: PID 25610

 Port Status:
   • Port 8002: ⚪ Available
   • Port 8050:  In Use
   • Port 8051: ⚪ Available
   • Port 8052: ⚪ Available
   •  Monitoring Services:  Active
   •  Active Dashboards:
     - http://localhost:8050

 RECOMMENDED NEXT STEPS:
   1.  Deploy models to different environments
   2.  Monitor model performance in production
   3.  Set up automated retraining
  

---

##  Congratulations!

You have successfully set up and executed the **Demand Stock Forecasting MLOps Pipeline**!

###  A Complete ML-Ops Process Accomplished:

 **Complete Data Pipeline** with validation and feature engineering  
 **Model Training & Registry** with automated versioning  
 **Multi-Environment Deployment** (dev/staging/production)  
 **Real-time Monitoring** with drift detection  
 **Quality Assurance** with comprehensive testing  
 **AWS Integration** with SageMaker, S3, and Athena  

###  Resources:

- **Project Repository**: https://github.com/btlambodh/demand-stock-forecasting-mlops
- **Author**: Bhupal Lambodhar (btiduwarlambodhar@sandiego.edu)
- **Institution**: University of San Diego

###  Keep Building!

This notebook provides a solid foundation for MLOps workflows. Continue enhancing with:
- Advanced model architectures
- Real-time feature engineering
- A/B testing capabilities
- Advanced monitoring and alerting
- Automated model retraining

---

*Happy MLOps!* 